In [1]:
from ozon_matching.kopatych_solution.utils import read_parquet, extract_category_levels
import polars as pl
from sklearn.model_selection import StratifiedKFold, train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from tqdm.auto import tqdm
import numpy as np
from matplotlib import pyplot as plt

In [2]:
data = pl.concat(
    [
        read_parquet('../data/test_data.parquet', columns=['variantid', 'categories']),
        read_parquet('../data/train_data.parquet', columns=['variantid', 'categories']),
    ]
)
data = data.unique(subset=['variantid'])

2023-05-23 18:48:56.221 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_data.parquet
2023-05-23 18:48:56.233 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 35730, N Cols - 2
2023-05-23 18:48:56.233 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_data.parquet
2023-05-23 18:48:56.348 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 457063, N Cols - 2


In [3]:
data = extract_category_levels(data, [3, 4])
data = data.select(pl.col(['variantid', 'category_level_3', 'category_level_4']))

In [4]:
data = data.join(
    (
        data
        .select(pl.col('category_level_3'))
        .unique()
        .with_row_count(name='category_level_3_id')
    ),
    on=['category_level_3']
)

data = data.join(
    (
        data
        .select(pl.col('category_level_4'))
        .unique()
        .with_row_count(name='category_level_4_id')
    ),
    on=['category_level_4']
)

data = data.drop(['category_level_3', 'category_level_4'])

In [15]:
order_columns = [
    'variantid1', 'variantid2', 'target', 'is_train'
]

train = read_parquet('../data/train_pairs.parquet')
train = train.with_columns(
    [
        pl.lit(1).cast(pl.Int8).alias('is_train'),
        pl.col('target').cast(pl.Int8).alias('target')
    ]
)

test = read_parquet('../data/test_pairs_wo_target.parquet')
test = test.drop(['__index_level_0__']).with_columns(
    [
        pl.lit(0).cast(pl.Int8).alias('is_train'),
        pl.lit(None).cast(pl.Int8).alias('target')
    ]
)

train = train.select(pl.col(order_columns))
test = test.select(pl.col(order_columns))

2023-05-25 08:07:57.869 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_pairs.parquet
2023-05-25 08:07:57.879 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 306540, N Cols - 3
2023-05-25 08:07:57.879 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:57 - shape: (5, 3)
┌────────┬────────────┬────────────┐
│ target ┆ variantid1 ┆ variantid2 │
│ ---    ┆ ---        ┆ ---        │
│ f64    ┆ i64        ┆ i64        │
╞════════╪════════════╪════════════╡
│ 0.0    ┆ 51197862   ┆ 51198054   │
│ 1.0    ┆ 53062686   ┆ 536165289  │
│ 1.0    ┆ 53602615   ┆ 587809782  │
│ 1.0    ┆ 53888651   ┆ 89598677   │
│ 0.0    ┆ 56930698   ┆ 551526166  │
└────────┴────────────┴────────────┘
2023-05-25 08:07:57.882 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_pairs_wo_target.parquet
2023-05-25 08:07:57.884 | INFO     | ozon_matching.kopatych_solution.utils:rea

In [16]:
train.head(1)

variantid1,variantid2,target,is_train
i64,i64,i8,i8
51197862,51198054,0,1


In [17]:
test.head(1)

variantid1,variantid2,target,is_train
i64,i64,i8,i8
52076340,290590137,null,0


In [18]:
pairs = pl.concat([train, test])

In [25]:
a = np.array(pairs['variantid1'].to_list())
b = np.array(pairs['variantid2'].to_list())

In [26]:
np.random.shuffle(a)
np.random.shuffle(b)

In [30]:
new_pairs = [
    [v1, v2]
    for v1, v2 in zip(a, b)
]

new_pairs = pl.DataFrame(new_pairs, orient='row', schema={'variantid1': pl.Int64, 'variantid2': pl.Int64})

In [33]:
new_pairs.join(
    pairs,
    on=['variantid1', 'variantid2'],
    how='left'
).filter(
    (pl.col('target').is_null()) & (pl.col('is_train').is_null())
).fill_null(0).drop(['is_train']).write_parquet('adversarial_v4.parquet')

In [34]:
np.quantile(np.random.rand(1000), 0.95)

0.9508266781518807

In [7]:
pairs = pairs.join(
    data.rename(
        {
            'variantid': 'variantid1',
            'category_level_3_id': 'category_level_3_id_1',
            'category_level_4_id': 'category_level_4_id_1',
        }
    ),
    on=['variantid1']
).join(
    data.rename(
        {
            'variantid': 'variantid2',
            'category_level_3_id': 'category_level_3_id_2',
            'category_level_4_id': 'category_level_4_id_2',
        }
    ),
    on=['variantid2']
).to_pandas()

In [8]:
18084

18084

In [9]:
import pandas as pd

In [10]:
cv1 = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
cv2 = StratifiedKFold(n_splits=5, random_state=13, shuffle=True)

X = pairs[
    [
        'category_level_3_id_1', 
        'category_level_4_id_1',
        'category_level_3_id_2', 
        'category_level_4_id_2'
    ]
].values
y = pairs['is_train'].values

adversarial_dataset = np.zeros((X.shape[0], 15))
adversarial_dataset[:] = np.nan

for n, (cv_index, holdout_index) in tqdm(enumerate(cv1.split(X, y))):
    X_cv, X_holdout = X[cv_index], X[holdout_index]
    y_cv, y_holdout = y[cv_index], y[holdout_index]
    models = []
    for m, (train_index, valid_index) in tqdm(enumerate(cv2.split(X_cv, y_cv))):
        X_train, X_valid = X_cv[train_index], X_cv[valid_index]
        y_train, y_valid = y_cv[train_index], y_cv[valid_index]

        model = LGBMClassifier(
            n_estimators=5000,
        )
        model.fit(
            X=X_train,
            y=y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric=['auc'],
            categorical_feature=[0,1,2,3],
            early_stopping_rounds=50
        )
        adversarial_dataset[holdout_index, m + n] = model.predict_proba(X_holdout)[:, 1]
        
pairs['adversarial'] = np.nanmean(adversarial_dataset, axis=1)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[1]	valid_0's auc: 0.786526	valid_0's binary_logloss: 0.207354
[2]	valid_0's auc: 0.788255	valid_0's binary_logloss: 0.202286
[3]	valid_0's auc: 0.788563	valid_0's binary_logloss: 0.198579
[4]	valid_0's auc: 0.788467	valid_0's binary_logloss: 0.195697
[5]	valid_0's auc: 0.788663	valid_0's binary_logloss: 0.193403
[6]	valid_0's auc: 0.789329	valid_0's binary_logloss: 0.191517
[7]	valid_0's auc: 0.789601	valid_0's binary_logloss: 0.189977
[8]	valid_0's auc: 0.789739	valid_0's binary_logloss: 0.188718
[9]	valid_0's auc: 0.790043	valid_0's binary_logloss: 0.187598
[10]	valid_0's auc: 0.790113	valid_0's binary_logloss: 0.186682
[11]	valid_0's auc: 0.790114	valid_0's binary_logloss: 0.185923
[12]	valid_0's auc: 0.790239	valid_0's binary_logloss: 0.185275
[13]	valid_0's auc: 0.790087	valid_0's binary_logloss: 0.184704
[14]	valid_0's auc: 0.790216	valid_0's binary_logloss: 0.184231
[15]	valid_0's auc: 0.790249	valid_0's binary_logloss: 0.183805
[16]	valid_0's auc: 0.790287	valid_0's binary_log

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

...

[1]	valid_0's auc: 0.776514	valid_0's binary_logloss: 0.207792
[2]	valid_0's auc: 0.778142	valid_0's binary_logloss: 0.203038
[3]	valid_0's auc: 0.779248	valid_0's binary_logloss: 0.19955
[4]	valid_0's auc: 0.780026	valid_0's binary_logloss: 0.196832
[5]	valid_0's auc: 0.779798	valid_0's binary_logloss: 0.194712
[6]	valid_0's auc: 0.779945	valid_0's binary_logloss: 0.192966
[7]	valid_0's auc: 0.780061	valid_0's binary_logloss: 0.191555
[8]	valid_0's auc: 0.780189	valid_0's binary_logloss: 0.190374
[9]	valid_0's auc: 0.780056	valid_0's binary_logloss: 0.189385
[10]	valid_0's auc: 0.780062	valid_0's binary_logloss: 0.188563
[11]	valid_0's auc: 0.780072	valid_0's binary_logloss: 0.187855
[12]	valid_0's auc: 0.780255	valid_0's binary_logloss: 0.187246
[13]	valid_0's auc: 0.780282	valid_0's binary_logloss: 0.186737
[14]	valid_0's auc: 0.780482	valid_0's binary_logloss: 0.186282
[15]	valid_0's auc: 0.7804	valid_0's binary_logloss: 0.185909
[16]	valid_0's auc: 0.780471	valid_0's binary_loglos

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[1]	valid_0's auc: 0.780625	valid_0's binary_logloss: 0.207549
[2]	valid_0's auc: 0.781825	valid_0's binary_logloss: 0.202717
[3]	valid_0's auc: 0.782165	valid_0's binary_logloss: 0.199134
[4]	valid_0's auc: 0.782518	valid_0's binary_logloss: 0.196381
[5]	valid_0's auc: 0.782582	valid_0's binary_logloss: 0.194182
[6]	valid_0's auc: 0.782681	valid_0's binary_logloss: 0.192405
[7]	valid_0's auc: 0.783131	valid_0's binary_logloss: 0.190917
[8]	valid_0's auc: 0.783519	valid_0's binary_logloss: 0.189686
[9]	valid_0's auc: 0.783687	valid_0's binary_logloss: 0.18864
[10]	valid_0's auc: 0.783833	valid_0's binary_logloss: 0.187752
[11]	valid_0's auc: 0.783892	valid_0's binary_logloss: 0.187009
[12]	valid_0's auc: 0.783822	valid_0's binary_logloss: 0.186367
[13]	valid_0's auc: 0.783989	valid_0's binary_logloss: 0.185802
[14]	valid_0's auc: 0.784064	valid_0's binary_logloss: 0.18532
[15]	valid_0's auc: 0.784093	valid_0's binary_logloss: 0.184924
[16]	valid_0's auc: 0.784177	valid_0's binary_loglo

/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/mikhailmarin/.cache/pypoetry/virtualenvs/ozon-matching-49GGzONq-py3.9/lib/python3.9/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3]
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/mikhailmarin/.cache/pypoetry/virtualenvs

[1]	valid_0's auc: 0.787706	valid_0's binary_logloss: 0.207627
[2]	valid_0's auc: 0.789272	valid_0's binary_logloss: 0.202574
[3]	valid_0's auc: 0.789734	valid_0's binary_logloss: 0.19891
[4]	valid_0's auc: 0.78952	valid_0's binary_logloss: 0.196099
[5]	valid_0's auc: 0.789782	valid_0's binary_logloss: 0.193821
[6]	valid_0's auc: 0.790008	valid_0's binary_logloss: 0.191941
[7]	valid_0's auc: 0.790469	valid_0's binary_logloss: 0.190398
[8]	valid_0's auc: 0.790412	valid_0's binary_logloss: 0.189115
[9]	valid_0's auc: 0.790708	valid_0's binary_logloss: 0.188028
[10]	valid_0's auc: 0.790622	valid_0's binary_logloss: 0.187093
[11]	valid_0's auc: 0.790608	valid_0's binary_logloss: 0.186305
[12]	valid_0's auc: 0.790469	valid_0's binary_logloss: 0.18562
[13]	valid_0's auc: 0.79058	valid_0's binary_logloss: 0.18502
[14]	valid_0's auc: 0.79076	valid_0's binary_logloss: 0.184498
[15]	valid_0's auc: 0.790697	valid_0's binary_logloss: 0.184064
[16]	valid_0's auc: 0.79081	valid_0's binary_logloss: 0

In [11]:
pl.from_pandas(
    pairs[pairs['is_train'] == 1].sort_values('adversarial').head(18084)[['variantid1', 'variantid2']]
).join(
    read_parquet('../data/train_pairs.parquet'),
    on=['variantid1', 'variantid2']
).write_parquet('../data/adversarial.parquet')

2023-05-23 18:49:49.853 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_pairs.parquet
2023-05-23 18:49:49.864 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 306540, N Cols - 3


In [12]:
read_parquet('../data/adversarial.parquet')

2023-05-23 19:00:54.775 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/adversarial.parquet
2023-05-23 19:00:54.778 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 18084, N Cols - 3


variantid1,variantid2,target
i64,i64,f64
85836778,85838082,0.0
85890302,85939258,1.0
85940318,148607642,1.0
85975076,508197093,1.0
89403250,90341703,0.0
89722250,177811799,1.0
100009852,750360172,1.0
129052413,177224924,1.0
159929290,553255415,0.0


In [ ]:
scores_cv

In [ ]:
np.mean(scores_cv)

In [ ]:
cv2 = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)

scores_cv = []

for fold_number in range(1, 6):
    pairs_test = pl.read_parquet(
        f'../experiments/v5/cv_{fold_number}/test/pairs.parquet', 
        columns=['variantid1', 'variantid2']
    )
    pairs_train = pl.read_parquet(
        f'../experiments/v5/cv_{fold_number}/train/pairs.parquet',
        columns=['variantid1', 'variantid2']
    )
    pairs_test = pairs_test.with_columns([pl.lit(0).cast(pl.Int8).alias('is_train')])
    pairs_train = pairs_train.with_columns([pl.lit(1).cast(pl.Int8).alias('is_train')])
    
    pairs = pl.concat([pairs_test, pairs_train])
    
    pairs = (
        pairs
        .join(
            data.rename(
                {
                    'variantid': 'variantid1',
                    'category_level_3_id': 'category_level_3_id_1',
                    'category_level_4_id': 'category_level_4_id_1',
                }
            ),
            on=['variantid1']
        )
        .join(
            data.rename(
                {
                    'variantid': 'variantid2',
                    'category_level_3_id': 'category_level_3_id_2',
                    'category_level_4_id': 'category_level_4_id_2',
                }
            ),
            on=['variantid2']
        )
        .drop(['variantid1', 'variantid2'])
        .to_pandas()
    )
    
    X = pairs.drop(columns=['is_train']).values
    y = pairs['is_train'].values
    
    X_cv, X_holdout, y_cv, y_holdout = train_test_split(X, y, test_size=0.2, random_state=13)
    for train_index, valid_index in tqdm(cv2.split(X_cv, y_cv)):
        X_train, X_valid = X_cv[train_index], X_cv[valid_index]
        y_train, y_valid = y_cv[train_index], y_cv[valid_index]

    
        model = LGBMClassifier(
            n_estimators=1000,
        )
        model.fit(
            X=X_train,
            y=y_train,
            eval_set=[(X_valid, y_valid)],
            eval_metric=['auc'],
            categorical_feature=[0,1,2,3],
            early_stopping_rounds=50
        )
        scores_cv.append(roc_auc_score(y_holdout, model.predict_proba(X_holdout)[:, 1]))

In [ ]:
np.mean(scores_cv)

In [ ]:
dataset = pl.read_parquet('../data/train_pairs.parquet')
dataset = dataset.join(
    data.rename(
        {
            'variantid': 'variantid1',
            'category_level_3_id': 'category_level_3_id_1',
            'category_level_4_id': 'category_level_4_id_1',
        }
    ),
    on=['variantid1']
).join(
    data.rename(
        {
            'variantid': 'variantid2',
            'category_level_3_id': 'category_level_3_id_2',
            'category_level_4_id': 'category_level_4_id_2',
        }
    ),
    on=['variantid2']
)

In [ ]:
dataset

In [ ]:
table = pl.DataFrame(schema={'variantid1': pl.Int64, 'variantid2': pl.Int64})
for fold_number in range(1, 6):
    for fold_type in ['train', 'test']:
        t = pl.read_csv(f'../experiments/v5/cv_{fold_number}/{fold_type}/prediction.csv')
        t = t.rename({'scores': f'fold_{fold_number}_{fold_type}'})
        table = table.join(t, on=['variantid1', 'variantid2'], how='outer')

In [ ]:
pl.read_parquet('../data/train_pairs.parquet').shape

In [ ]:
from ozon_matching.kopatych_solution.utils import write_parquet
write_parquet(table, '../experiments/v5/cv_pivot.parquet')

In [13]:
cats = np.random.randint(1, 100, 10000)
unique_cats, counts = np.unique(cats, return_counts=True)
unique_cats

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [14]:
counts

array([129, 111,  93, 112, 104,  98, 102,  88,  99,  98, 103, 116, 104,
        93, 111, 111, 103,  88, 105,  96, 104, 115, 102,  93,  86, 103,
       116,  83, 118,  89,  87, 105,  96, 108, 112, 108, 108,  98,  85,
        82, 102,  88, 106,  86,  94,  93,  95,  83, 101, 102,  93, 100,
        89, 110, 113,  94,  92,  95,  87,  92, 101, 119, 104, 102,  82,
       109,  92, 118,  99, 106, 104, 119, 102,  67, 111,  97,  90, 107,
       123, 120, 107, 116, 105,  98,  98, 109, 131,  98,  86, 119, 111,
        95,  91,  96, 104,  95,  94, 105,  93])

In [15]:
data = pl.concat(
    [
        read_parquet('../data/test_data.parquet', columns=['variantid', 'categories']),
        read_parquet('../data/train_data.parquet', columns=['variantid', 'categories']),
    ]
)
data = data.unique(subset=['variantid'])

2023-05-23 21:12:35.328 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_data.parquet
2023-05-23 21:12:35.336 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 35730, N Cols - 2
2023-05-23 21:12:35.336 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_data.parquet
2023-05-23 21:12:35.459 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 457063, N Cols - 2


In [16]:
data = extract_category_levels(data, [3, 4])
data = data.select(pl.col(['variantid', 'category_level_3', 'category_level_4']))

In [17]:
data = data.join(
    (
        data
        .select(pl.col('category_level_3'))
        .unique()
        .with_row_count(name='category_level_3_id')
    ),
    on=['category_level_3']
)

data = data.join(
    (
        data
        .select(pl.col('category_level_4'))
        .unique()
        .with_row_count(name='category_level_4_id')
    ),
    on=['category_level_4']
)

data = data.drop(['category_level_3', 'category_level_4'])

In [18]:
train = read_parquet('../data/train_pairs.parquet')
train = train.drop(['target']).with_columns([pl.lit(1).cast(pl.Int8).alias('is_train')])

test = read_parquet('../data/test_pairs_wo_target.parquet')
test = test.drop(['__index_level_0__']).with_columns([pl.lit(0).cast(pl.Int8).alias('is_train')])

2023-05-23 21:12:36.660 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/train_pairs.parquet
2023-05-23 21:12:36.672 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 306540, N Cols - 3
2023-05-23 21:12:36.674 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:54 - Read Parquet from ../data/test_pairs_wo_target.parquet
2023-05-23 21:12:36.677 | INFO     | ozon_matching.kopatych_solution.utils:read_parquet:56 - N Rows - 18084, N Cols - 3


In [19]:
pairs = pl.concat([train, test])

In [20]:
pairs = pairs.join(
    data.rename(
        {
            'variantid': 'variantid1',
            'category_level_3_id': 'category_level_3_id_1',
            'category_level_4_id': 'category_level_4_id_1',
        }
    ),
    on=['variantid1']
).join(
    data.rename(
        {
            'variantid': 'variantid2',
            'category_level_3_id': 'category_level_3_id_2',
            'category_level_4_id': 'category_level_4_id_2',
        }
    ),
    on=['variantid2']
)

In [21]:
pairs

variantid1,variantid2,is_train,category_level_3_id_1,category_level_4_id_1,category_level_3_id_2,category_level_4_id_2
i64,i64,i8,u32,u32,u32,u32
91616256,91618048,1,87,120,87,120
195867339,195996128,1,43,329,43,329
532527673,538263776,1,47,352,47,352
795470824,795525568,1,68,208,68,208
93805858,190378432,1,68,208,68,208
190373379,190378432,1,68,208,68,208
692284332,721805472,1,70,198,70,198
256395898,487503808,0,72,155,72,34
234014245,234027232,1,22,278,22,278
